# IndicF5 Streaming TTS Demo

**Features:** 🎧 Streaming audio - plays progressively!

**Setup:** T4 GPU + HF access to https://huggingface.co/ai4bharat/IndicF5

In [ ]:
import torch
assert torch.cuda.is_available(), '❌ GPU!'
print(f'✅ GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# RESTART RUNTIME AFTER THIS
!pip uninstall -y numpy scipy -q
!pip install numpy==1.26.4 scipy -q
!pip install 'transformers<4.50' accelerate -q
!pip install git+https://github.com/ai4bharat/IndicF5.git -q
!pip install gradio torchcodec soundfile requests -q
print('⚠️ RESTART RUNTIME!')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch, gradio as gr, tempfile, soundfile as sf, numpy as np, requests, io, re, os
from transformers import AutoModel

def load_audio_url(url):
    r = requests.get(url)
    data, sr = sf.read(io.BytesIO(r.content))
    return sr, (data * 32768).astype(np.int16) if data.dtype == np.float64 else data

def split_sentences(text):
    parts = re.split(r'[.!?।॥\n]+', text)
    return [p.strip() for p in parts if p.strip()]

# ALL ORIGINAL EXAMPLES
EXAMPLES = [
    {
        'name': 'PAN_F (Happy)',
        'url': 'https://github.com/AI4Bharat/IndicF5/raw/refs/heads/main/prompts/PAN_F_HAPPY_00002.wav',
        'ref_text': 'ਇੱਕ ਗ੍ਰਾਹਕ ਨੇ ਸਾਡੀ ਬੇਮਿਸਾਲ ਸੇਵਾ ਬਾਰੇ ਦਿਲੋਂਗਵਾਹੀ ਦਿੱਤੀ ਜਿਸ ਨਾਲ ਸਾਨੂੰ ਅਨੰਦ ਮਹਿਸੂਸ ਹੋਇਆ।',
        'synth': 'मैं बिना किसी चिंता के अपने दोस्तों को अपने ऑटोमोबाइल एक्सपर्ट के पास भेज देता हूँ क्योंकि मैं जानता हूँ कि वह निश्चित रूप से उनकी सभी जरूरतों पर खरा उतरेगा।'
    },
    {
        'name': 'TAM_F (Happy)',
        'url': 'https://github.com/AI4Bharat/IndicF5/raw/refs/heads/main/prompts/TAM_F_HAPPY_00001.wav',
        'ref_text': 'நான் நெனச்ச மாதிரியே அமேசான்ல பெரிய தள்ளுபடி வந்திருக்கு. கம்மி காசுக்கே அந்தப் புது சேம்சங் மாடல வாங்கிடலாம்.',
        'synth': 'ഭക്ഷണത്തിന് ശേഷം തൈര് സാദം കഴിച്ചാൽ ഒരു ഉഷാറാണ്!'
    },
    {
        'name': 'MAR_F (WIKI)',
        'url': 'https://github.com/AI4Bharat/IndicF5/raw/refs/heads/main/prompts/MAR_F_WIKI_00001.wav',
        'ref_text': 'दिगंटराव्दारे अंतराळ कक्षेतला कचरा चिन्हित करण्यासाठी प्रयत्न केले जात आहे.',
        'synth': 'प्रारंभिक अंकुर छेदक. मी सोलापूर जिल्ह्यातील माळशिरस तालुक्यातील शेतकरी गणपत पाटील बोलतोय. माझ्या ऊस पिकावर प्रारंभिक अंकुर छेदक कीड आढळत आहे. क्लोरँट्रानिलीप्रोल (कोराजेन) वापरणे योग्य आहे का? त्याचे प्रमाण किती असावे?'
    },
    {
        'name': 'MAR_M (WIKI)',
        'url': 'https://github.com/AI4Bharat/IndicF5/raw/refs/heads/main/prompts/MAR_M_WIKI_00001.wav',
        'ref_text': 'या प्रथाला एकोणीसशे पंचातर ईसवी पासून भारतीय दंड संहिताची धारा चारशे अठ्ठावीस आणि चारशे एकोणतीसच्या अन्तर्गत निषेध केला.',
        'synth': 'जीवाणू करपा. मी अहमदनगर जिल्ह्यातील राहुरी गावातून बाळासाहेब जाधव बोलतोय. माझ्या डाळिंब बागेत जीवाणू करपा मोठ्या प्रमाणात दिसतोय. स्ट्रेप्टोसायक्लिन आणि कॉपर ऑक्सिक्लोराईड फवारणीसाठी योग्य प्रमाण काय असावे?'
    },
    {
        'name': 'KAN_F (Happy)',
        'url': 'https://github.com/AI4Bharat/IndicF5/raw/refs/heads/main/prompts/KAN_F_HAPPY_00001.wav',
        'ref_text': 'ನಮ್‌ ಫ್ರಿಜ್ಜಲ್ಲಿ  ಕೂಲಿಂಗ್‌ ಸಮಸ್ಯೆ ಆಗಿ ನಾನ್‌ ಭಾಳ ದಿನದಿಂದ ಒದ್ದಾಡ್ತಿದ್ದೆ, ಆದ್ರೆ ಅದ್ನೀಗ ಮೆಕಾನಿಕ್ ಆಗಿರೋ ನಿಮ್‌ ಸಹಾಯ್ದಿಂದ ಬಗೆಹರಿಸ್ಕೋಬೋದು ಅಂತಾಗಿ ನಿರಾಳ ಆಯ್ತು ನಂಗೆ.',
        'synth': 'চেন্নাইয়ের শেয়ারের অটোর যাত্রীদের মধ্যে খাবার ভাগ করে খাওয়াটা আমার কাছে মন খুব ভালো করে দেওয়া একটা বিষয়।'
    },
]

print('Loading examples...')
for ex in EXAMPLES:
    ex['sr'], ex['data'] = load_audio_url(ex['url'])
print(f'✅ {len(EXAMPLES)} examples loaded')

print('Loading IndicF5...')
model = AutoModel.from_pretrained('ai4bharat/IndicF5', trust_remote_code=True).to('cuda')
print('✅ Model ready!')

# STREAMING: Yields individual WAV chunks
def synthesize_streaming(text, ref_audio, ref_text):
    if not text or ref_audio is None or not ref_text:
        return
    
    sr, data = ref_audio
    sentences = split_sentences(text)
    print(f'[STREAM] {len(sentences)} sentences')
    
    ref_file = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
    sf.write(ref_file.name, data, sr)
    
    for i, sentence in enumerate(sentences):
        print(f'[{i+1}/{len(sentences)}] {sentence[:40]}...')
        chunk = model(sentence, ref_audio_path=ref_file.name, ref_text=ref_text)
        
        if chunk.dtype == np.int16:
            chunk = chunk.astype(np.float32) / 32768.0
        
        chunk_file = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
        sf.write(chunk_file.name, chunk, 24000)
        yield chunk_file.name
    
    print('[STREAM] Done!')

def load_example(name):
    ex = next((e for e in EXAMPLES if e['name'] == name), None)
    return ((ex['sr'], ex['data']), ex['ref_text'], ex['synth']) if ex else (None, '', '')

# Build UI with examples table like original
with gr.Blocks(title='IndicF5 Streaming') as app:
    gr.Markdown('# 🎧 IndicF5 Streaming TTS')
    gr.Markdown('Choose an example or provide your own inputs:')
    
    # Examples dropdown
    dd = gr.Dropdown([e['name'] for e in EXAMPLES], label='📂 Choose an example')
    
    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label='Text to Synthesize', lines=4)
            ref = gr.Audio(label='Reference Prompt Audio', type='numpy')
            ref_txt = gr.Textbox(label='Text in Reference Prompt Audio', lines=2)
            btn = gr.Button('🎤 Generate (Streaming)', variant='primary')
        
        out = gr.Audio(label='Generated Audio (streams progressively)', streaming=True, autoplay=True)
    
    # Examples table for reference
    gr.Markdown('### Available Examples:')
    examples_data = [[e['name'], e['synth'][:60]+'...', e['ref_text'][:60]+'...'] for e in EXAMPLES]
    gr.Dataframe(value=examples_data, headers=['Voice', 'Text to Synthesize', 'Reference Text'], interactive=False)
    
    dd.change(load_example, [dd], [ref, ref_txt, txt])
    btn.click(synthesize_streaming, [txt, ref, ref_txt], [out])

app.launch(share=True, debug=True)